In [1]:
import pandapower as pp
import pandapower.networks
import numpy as np
import pandas as pd

In [2]:
#if network == 'IEEE-14' then
net = pp.networks.case14()
#else if network == 'IEEE-30' then
#net = pandapower.networks.case30()
#else if network == 'IEEE-57' then
#net = pandapower.networks.case57()
#else if network == 'IEEE-118' then
#net = pandapower.networks.case118()

In [15]:
# if user selects data generation strategy 1
pp.runpp(net)
# state vector x_base (which is again (number of nodes, ) but a complex vector)
x_base = net.res_bus.vm_pu.to_numpy()*np.exp(1j*net.res_bus.va_degree.to_numpy()/180*np.pi)
# number of discrete time steps
T = 100 # user argument
# Dx ~ N(0,Cov_Mat)
mean = np.zeros(x_base.shape)
cov = np.eye(x_base.shape[0])/1000 # to be refined in the future
delta_x_mat = np.transpose(np.random.multivariate_normal(mean, cov**2, size=T))
x_base_mat = np.repeat(x_base.reshape((x_base.shape[0],-1)), T, axis=1)
x_t_mat = x_base_mat + delta_x_mat

numba cannot be imported and numba functions are disabled.
Probably the execution is slow.
Please install numba to gain a massive speedup.
(or if you prefer slow execution, set the flag numba=False to avoid this warning!)



In [14]:
# Sf = V[ branch[br, F_BUS].astype(int) ] * conj(Yf[br, :] * V)
Sf = []
for i in range(0,net._ppc['branch'].shape[0]): # to be refined
    Sf.append(x_t_mat[net._ppc['branch'][i, 0].astype(int)]*np.conj(net._ppc['internal']['Yf'][i,:]*x_t_mat))
    #St.append(x_base[net._ppc['branch'][i, 1].astype(int)]*np.conj(net._ppc['internal']['Yt'][i,:]*x_base))
Sf = np.asarray(Sf).reshape((net._ppc['branch'].shape[0],-1))
z_P = np.real(Sf)
z_Q = np.imag(Sf)
z_V_mag = np.abs(x_t_mat[0,:])
z_t_mat = np.vstack((z_P,z_Q))
z_t_mat = np.vstack((z_t_mat,z_V_mag))
# in the DC case, we had 20 measurements since z_P (20 x 1)
# in the AC case, we have 20 from z_P, 20 from z_Q, and 1 from z_V_mag -> 41

C:\Users\pvedi\AppData\Local\Temp\ipykernel_17048\2541989653.py:4: ComplexWarning: Casting complex values to real discards the imaginary part
  Sf.append(x_t_mat[net._ppc['branch'][i, 0].astype(int)]*np.conj(net._ppc['internal']['Yf'][i,:]*x_t_mat))


In [7]:
from pandapower.pypower.dSbr_dV import dSbr_dV
import scipy as sc

In [8]:
# AC State Estimator

# in DC state estimator: x_est = inv(H.T @ H) @ H.T @ z

# in DC case: x = [theta_0, theta_1, ...]
# in AC case: x = [theta_0, theta_1, ..., theta_13, mag_0, mag_1, ..., mag_13]
# we need theta_0 = 0

# initial guess for state vector
x_est = np.ones((len(net.bus)))
# initially, all voltage magnitudes are assumed to be 1 and all voltage angles are assumed to 0

residuals = 1

# z = H*x + e -> z = h(x) + e

count = 0
while np.max(np.abs(residuals)) > 1e-2:

    count += 1
    print(count)
    # h(x_est) for the current x_est (which is just x_hat), equivalent to H*x_hat
    Sf_x_est = []
    for i in range(0,net._ppc['branch'].shape[0]): # to be refined
        Sf_x_est.append(x_est[net._ppc['branch'][i, 0].astype(int)]*np.conj(net._ppc['internal']['Yf'][i,:]*x_est))
    Sf_x_est = np.asarray(Sf_x_est).reshape((net._ppc['branch'].shape[0],-1))    
    P_x_est = np.real(Sf_x_est)
    Q_x_est = np.imag(Sf_x_est)
    V_x_est = x_est[0]
    z_x_est = np.vstack((P_x_est,Q_x_est)).reshape(-1)
    z_x_est = np.hstack((z_x_est,V_x_est))
    
    residuals = z_t_mat[:,0] - z_x_est
    dSf_dVa, dSf_dVm, _, _, _, _ = dSbr_dV(net._ppc['branch'], net._ppc['internal']['Yf'], net._ppc['internal']['Yt'], x_est)
    dV_dVa = 1j * np.diag(x_est)
    dV_dVa = sc.sparse.csr_matrix(dV_dVa[0,:])
    dV_dVm = np.diag(x_est/np.abs(x_est))
    dV_dVm = sc.sparse.csr_matrix(dV_dVm[0,:])
    
    H_P = sc.sparse.hstack((np.real(dSf_dVa),np.real(dSf_dVm)))
    H_Q = sc.sparse.hstack((np.imag(dSf_dVa),np.imag(dSf_dVm)))
    H_V = sc.sparse.hstack((dV_dVa,dV_dVm))
    H = sc.sparse.vstack((H_P, H_Q))
    H = sc.sparse.vstack((H, H_V))
    H = sc.sparse.csr_matrix(H)[:,1:]
    
    delta_x_est = sc.sparse.linalg.spsolve(np.transpose(H) @ H, np.transpose(H) @ residuals)
    delta_x_est_a = delta_x_est[0:net._ppc['bus'].shape[0]-1]
    delta_x_est_m = delta_x_est[net._ppc['bus'].shape[0]-1:]

    x_est_a = np.angle(x_est[1:]) + delta_x_est_a
    x_est_a = np.hstack((0,x_est_a))
    x_est_m = np.abs(x_est) + delta_x_est_m
    x_est = x_est_m*np.exp(1j*x_est_a)
    
# the AC estimator returns the final x_est, H, z_x_est

1
2
3


C:\Users\pvedi\AppData\Local\Temp\ipykernel_17048\362787764.py:25: ComplexWarning: Casting complex values to real discards the imaginary part
  Sf_x_est.append(x_est[net._ppc['branch'][i, 0].astype(int)]*np.conj(net._ppc['internal']['Yf'][i,:]*x_est))


In [7]:
# for installing idaes
# https://ndcbe.github.io/CBE60499/01.00-Pyomo-Introduction.html#1.0.1.3-Install-IDAES-PSE
# https://idaes-pse.readthedocs.io/en/latest/tutorials/getting_started/index.html

In [8]:
# def compute_h(x_est, net)
# to be used in residual computation for anomaly detection where:
# [z_i - h(x)_i]/cov_ii instead of [z_i - [H*x]_i]/cov_ii where x is the final estimate of the AC estimator

Z = net._ppc['branch'][:,2] + 1j*net._ppc['branch'][:,3]
C = 1/Z

tap = np.real(net._ppc['branch'][:,8])

g = np.real(C)
b = np.imag(C)

bs_line = net._ppc['branch'][:,4].astype(float)/2
#bs_shunt = -net._ppc['bus'][:,5]/net._ppc['baseMVA']

Pij = []
Qij = []
for i in range(0, net._ppc['branch'].shape[0]):
    Pij.append(np.abs(x_t_mat[net._ppc['branch'][i, 0].astype(int)])**2 * g[i]/tap[i]**2 
               - np.abs(x_t_mat[net._ppc['branch'][i, 0].astype(int)])*np.abs(x_t_mat[net._ppc['branch'][i, 1].astype(int)])
               *(g[i]*np.cos(np.angle(x_t_mat[net._ppc['branch'][i, 0].astype(int)])-np.angle(x_t_mat[net._ppc['branch'][i, 1].astype(int)]))
                 + b[i]*np.sin(np.angle(x_t_mat[net._ppc['branch'][i, 0].astype(int)])-np.angle(x_t_mat[net._ppc['branch'][i, 1].astype(int)])))/tap[i])

    Qij.append(-np.abs(x_t_mat[net._ppc['branch'][i, 0].astype(int)])**2 * ((b[i] + bs_line[i])/tap[i]**2)
               - np.abs(x_t_mat[net._ppc['branch'][i, 0].astype(int)])*np.abs(x_t_mat[net._ppc['branch'][i, 1].astype(int)])
               *(g[i]*np.sin(np.angle(x_t_mat[net._ppc['branch'][i, 0].astype(int)])-np.angle(x_t_mat[net._ppc['branch'][i, 1].astype(int)]))
                 - b[i]*np.cos(np.angle(x_t_mat[net._ppc['branch'][i, 0].astype(int)])-np.angle(x_t_mat[net._ppc['branch'][i, 1].astype(int)])))/tap[i])

Pij = np.asarray(Pij).reshape((net._ppc['branch'].shape[0],-1))
Qij = np.asarray(Qij).reshape((net._ppc['branch'].shape[0],-1))
  
V_x_est = np.abs(x_est[0])
z_x_est = np.vstack((Pij,Qij)).reshape(-1)
z_x_est = np.hstack((z_x_est,V_x_est))

# returns z_x_est which is h(x_est)

<ipython-input-8-4aa50a25da3c>:13: ComplexWarning: Casting complex values to real discards the imaginary part
  bs_line = net._ppc['branch'][:,4].astype(float)/2
<ipython-input-8-4aa50a25da3c>:19: ComplexWarning: Casting complex values to real discards the imaginary part
  Pij.append(np.abs(x_t_mat[net._ppc['branch'][i, 0].astype(int)])**2 * g[i]/tap[i]**2
<ipython-input-8-4aa50a25da3c>:20: ComplexWarning: Casting complex values to real discards the imaginary part
  - np.abs(x_t_mat[net._ppc['branch'][i, 0].astype(int)])*np.abs(x_t_mat[net._ppc['branch'][i, 1].astype(int)])
<ipython-input-8-4aa50a25da3c>:21: ComplexWarning: Casting complex values to real discards the imaginary part
  *(g[i]*np.cos(np.angle(x_t_mat[net._ppc['branch'][i, 0].astype(int)])-np.angle(x_t_mat[net._ppc['branch'][i, 1].astype(int)]))
<ipython-input-8-4aa50a25da3c>:22: ComplexWarning: Casting complex values to real discards the imaginary part
  + b[i]*np.sin(np.angle(x_t_mat[net._ppc['branch'][i, 0].astype(int)]

In [9]:
import pyomo.environ as pyo
import idaes

k = 1
delta_a = 0.1 # in reality, a change by 0.1 is 0.1*100 MW = 10 MW
a_lower_bound, a_upper_bound, c_lower_bound, c_upper_bound = -1000, 1000, -1000, 1000

m = pyo.ConcreteModel()

m.a_num = range(H.shape[0])
m.c_num = range(H.shape[1]+1)
m.a = pyo.Var(m.a_num, domain=pyo.Reals, initialize=0)
m.c = pyo.Var(m.c_num, domain=pyo.Reals, initialize=0)
m.a_pos = pyo.Var(m.a_num, domain=pyo.NonNegativeReals, initialize=0)
m.a_neg = pyo.Var(m.a_num, domain=pyo.NonPositiveReals, initialize=0)

m.abs_a = pyo.ConstraintList()
for i in m.a_num:
    m.abs_a.add(m.a[i] == m.a_pos[i]+m.a_neg[i]) 
    
def compute_h_m(x_est, m, net):  
    # x_est is the final estimate for a given point in time, x_hat[:,t]
    #m.i_aux = pyo.Param(initialize=-1, mutable=True)
    #m.i = pyo.Expression(expr = m.i_aux**0.5)    
    x_est_a = np.asarray([np.angle(x_est[i]) + m.c[i] for i in range(0,net._ppc['bus'].shape[0])])
    x_est_m = np.asarray([np.abs(x_est[i-net._ppc['bus'].shape[0]]) + m.c[i] for i in range(net._ppc['bus'].shape[0],2*net._ppc['bus'].shape[0])])
    #x_est = x_est_m*np.exp(1j*x_est_a)
    #x_est = x_est_m*pyo.exp(m.i*x_est_a)
    #m.x_est = [x_est_m[i]*pyo.exp(m.i*x_est_a[i]) for i in range(0,net._ppc['bus'].shape[0])]
    
    #Sf_x_est = []
    #for i in range(0,net._ppc['branch'].shape[0]): # to be refined
    #    Sf_x_est.append(x_est[net._ppc['branch'][i, 0].astype(int)]*np.conj(net._ppc['internal']['Yf'][i,:]*x_est))
    #Sf_x_est = np.asarray(Sf_x_est).reshape((net._ppc['branch'].shape[0],-1))    
    #P_x_est = np.real(Sf_x_est)
    #Q_x_est = np.imag(Sf_x_est)

    Z = net._ppc['branch'][:,2] + 1j*net._ppc['branch'][:,3]
    C = 1/Z

    tap = np.real(net._ppc['branch'][:,8])

    g = np.real(C)
    b = np.imag(C)

    bs_line = net._ppc['branch'][:,4].astype(float)/2

    Pij = []
    Qij = []
    for i in range(0, net._ppc['branch'].shape[0]):
        Pij.append(x_est_m[net._ppc['branch'][i, 0].astype(int)]**2 * g[i]/tap[i]**2 
                   - x_est_m[net._ppc['branch'][i, 0].astype(int)]*x_est_m[net._ppc['branch'][i, 1].astype(int)]
                   *(g[i]*pyo.cos(x_est_a[net._ppc['branch'][i, 0].astype(int)]-x_est_a[net._ppc['branch'][i, 1].astype(int)])
                     + b[i]*pyo.sin(x_est_a[net._ppc['branch'][i, 0].astype(int)]-x_est_a[net._ppc['branch'][i, 1].astype(int)]))/tap[i])

        Qij.append(-x_est_m[net._ppc['branch'][i, 0].astype(int)]**2 * ((b[i] + bs_line[i])/tap[i]**2)
                   - x_est_m[net._ppc['branch'][i, 0].astype(int)]*x_est_m[net._ppc['branch'][i, 1].astype(int)]
                   *(g[i]*pyo.sin(x_est_a[net._ppc['branch'][i, 0].astype(int)]-x_est_a[net._ppc['branch'][i, 1].astype(int)])
                     - b[i]*pyo.cos(x_est_a[net._ppc['branch'][i, 0].astype(int)]-x_est_a[net._ppc['branch'][i, 1].astype(int)]))/tap[i])

    Pij = np.asarray(Pij).reshape((net._ppc['branch'].shape[0],-1))
    Qij = np.asarray(Qij).reshape((net._ppc['branch'].shape[0],-1))    
    
    V_x_est = x_est_m[0]
    z_x_est = np.vstack((Pij,Qij)).reshape(-1)
    z_x_est = np.hstack((z_x_est,V_x_est))
    # z_x_est is h(x+c)
    return z_x_est 

# a = h(x_hat+c) - h(x_hat) instead of a = Hc, where x_hat is the output of AC estimator

m.stealthy_a = pyo.ConstraintList()
for i in m.a_num:
    # z_x_est is the h(x_hat) from the output of the AC estimator
    # compute_h_m(x_est, m, net) is the h(x_hat + c), where c is a variable vector
    m.stealthy_a.add(m.a[i] == compute_h_m(x_est, m, net)[i] - np.real(z_x_est[i]))

m.change_a = pyo.Constraint(expr = (m.a[k] == delta_a))

m.change_c = pyo.Constraint(expr = (m.c[0] == 0))
    
m.bounds_a = pyo.ConstraintList()
for i in m.a_num:
    m.bounds_a.add(m.a[i] <= a_upper_bound)
    m.bounds_a.add(m.a[i] >= a_lower_bound)

m.bounds_c = pyo.ConstraintList()
for i in m.c_num:
    m.bounds_c.add(m.c[i] <= c_upper_bound)
    m.bounds_c.add(m.c[i] >= c_lower_bound)

def abs_a_val(m):
    return sum(m.a_pos[i] - m.a_neg[i] for i in m.a_num)
    
m.value = pyo.Objective(rule=abs_a_val, sense = pyo.minimize)    
    
#m.pprint()

optimizer = pyo.SolverFactory('ipopt')
#optimizer = pyo.SolverFactory('gurobi')
status_opti = optimizer.solve(m, tee=True)

a_output = np.asarray([pyo.value(m.a[i]) for i in m.a_num])
c_output = np.asarray([pyo.value(m.c[i]) for i in m.c_num])

C:\Users\pvedi\AppData\Local\Temp\ipykernel_17048\1043828893.py:46: ComplexWarning: Casting complex values to real discards the imaginary part
  bs_line = net._ppc['branch'][:,4].astype(float)/2
C:\Users\pvedi\AppData\Local\Temp\ipykernel_17048\1043828893.py:51: ComplexWarning: Casting complex values to real discards the imaginary part
  Pij.append(x_est_m[net._ppc['branch'][i, 0].astype(int)]**2 * g[i]/tap[i]**2
C:\Users\pvedi\AppData\Local\Temp\ipykernel_17048\1043828893.py:52: ComplexWarning: Casting complex values to real discards the imaginary part
  - x_est_m[net._ppc['branch'][i, 0].astype(int)]*x_est_m[net._ppc['branch'][i, 1].astype(int)]
C:\Users\pvedi\AppData\Local\Temp\ipykernel_17048\1043828893.py:53: ComplexWarning: Casting complex values to real discards the imaginary part
  *(g[i]*pyo.cos(x_est_a[net._ppc['branch'][i, 0].astype(int)]-x_est_a[net._ppc['branch'][i, 1].astype(int)])
C:\Users\pvedi\AppData\Local\Temp\ipykernel_17048\1043828893.py:54: ComplexWarning: Casting

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt

This version of Ipopt was compiled from source code available at
    https://github.com/IDAES/Ipopt as part of the Institute for the Design of
    Advanced Energy Systems Process Systems Engineering Framework (IDAES PSE
    Framework) Copyright (c) 2018-2019. See https://github.com/IDAES/idaes-pse.

This version of Ipopt was compiled using HSL, a collection of Fortran codes
    for large-scale scientific computation.  All technical papers, sales and
    publicity material resulting from use of the HSL codes within IPOPT must
    contain the following acknowledgement:
        HSL, a collection of Fortran codes for large-scale scientific
        computation. See http://

In [10]:
status_opti #['Solver']

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 222, 'Number of variables': 151, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.13.2\\x3a Optimal Solution Found', 'Termination condition': 'optimal', 'Id': 0, 'Error rc': 0, 'Time': 0.0783376693725586}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [10]:
z_x_est.shape

(41,)

In [11]:
H.shape

(41, 27)

In [12]:
x_est.shape

(14,)